In [34]:
import requests
import json
import time
import os
from dotenv import load_dotenv

# API_KEY = "Your google API key here. if on development environments"

# else create a .env file using python dotenv package and load as below
load_dotenv()
API_KEY = os.getenv("API_KEY")
PLACES_API_BASE_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
DETAILS_API_BASE_URL = "https://maps.googleapis.com/maps/api/place/details/json"

In [35]:
def fetch_place_ids_with_pagination(location, radius, keyword):
    place_ids = []
    next_page_token = None

    while True:
        params = {
            "key": API_KEY,
            "location": location,
            "radius": radius,
            "keyword": keyword,
            "pagetoken": next_page_token if next_page_token else None
        }
        response = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params=params)
        if response.status_code == 200:
            data = response.json()
            print(data)  # Debug: Print the API response
            results = data.get("results", [])
            place_ids.extend([place["place_id"] for place in results])

            next_page_token = data.get("next_page_token")
            if not next_page_token:
                break
        else:
            print(f"Error fetching place IDs: {response.status_code} - {response.text}")
            break

    return place_ids

In [36]:
def get_place_details(place_ids):
    place_details = []
    
    for place_id in place_ids:
        place_url = f"https://maps.googleapis.com/maps/api/place/details/json"
        params = {
            "key": API_KEY,
            "place_id": place_id,
            "fields": "name,formatted_address,website,review,photos,formatted_phone_number,types"
        }
        
        # Fetch place details
        response = requests.get(place_url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            place_info = data.get("result", {})
            
            # Generate photo URLs
            photos = place_info.get("photos", [])
            photo_urls = [
                f"https://maps.googleapis.com/maps/api/place/photo?maxwidth=800&photoreference={photo['photo_reference']}&key={API_KEY}"
                for photo in photos
            ]
            
            details = {
                "name": place_info.get("name"),
                "address": place_info.get("formatted_address"),
                "website": place_info.get("website"),
                "reviews": place_info.get("reviews", []),
                "photos": photo_urls,
                "phone_number": place_info.get("formatted_phone_number"),
                "category": place_info.get("types", [])  # Fetching category (types)
            }
            place_details.append(details)
        else:
            print(f"Error {response.status_code}: {response.text}")
    
    return place_details


In [37]:
def get_places_with_details(location, radius, keyword):
    """
    Fetch place details including address, website, reviews, images, and phone numbers.
    Filters places with reviews rated 4.5 or above.
    """
    place_ids = fetch_place_ids_with_pagination(location, radius, keyword)

    all_places = []
    for place_id in place_ids:
        details = get_place_details([place_id])
        if details:
            # Collect place details
            place_data = {
                "name": details[0].get("name", "N/A"),
                "website": details[0].get("website", "N/A"),
                "address": details[0].get("address", "N/A"),
                "phone_number": details[0].get("phone_number", "N/A"),
                "reviews": [
                    {
                        "author": review.get("author_name", "Anonymous"),
                        "text": review.get("text", ""),
                        "rating": review.get("rating", "N/A")
                    }
                    for review in details[0].get("reviews", []) if review.get("rating", 0) >= 4.5
                ],
                "photos": [
                    f"https://maps.googleapis.com/maps/api/place/photo?maxwidth=800&photoreference={photo['photo_reference']}&key={API_KEY}"
                    for photo in details[0].get("photos", [])
                ],  # Generate photo URLs
                "category": details[0].get("category", "N/A"),  # Place category (types)
            }
            all_places.append(place_data)

            # Display data
            print(f"Name: {place_data['name']}")
            print(f"Website: {place_data['website']}")
            print(f"Address: {place_data['address']}")
            print(f"Phone Number: {place_data['phone_number']}")
            print("High-Rated Reviews (4.5+):")
            for review in place_data["reviews"]:
                print(f"  - {review['author']}: {review['text']} (Rating: {review['rating']})")
            print("Photos:")
            for photo in place_data["photos"]:
                print(f"  - {photo}")
            print(f"Category: {place_data['category']}")
            print("-" * 50)

    return all_places


In [38]:
def save_to_json(data, filename="places_with_reviews.json"):
    """
    Save the place details to a JSON file.
    """
    if data:
        with open(filename, "w") as file:
            json.dump(data, file, indent=4)
        print(f"Data saved to {filename} with {len(data)} places.")
    else:
        print("No data to save.")

In [39]:
# Example location (latitude,longitude for Miami, FL)
location = "25.761681,-80.191788"  # Miami, FL
radius = 50000  # Maximum allowed radius (50km)
keyword = "Plastic surgery recovery house"  # Search keyword

# Fetch place IDs with pagination
place_ids = fetch_place_ids_with_pagination(location, radius, keyword)

# Fetch details for each place
places = get_place_details(place_ids)

# Display total number of places fetched
total_places = len(places)
if total_places > 0:
    print(f"Total places fetched: {total_places}")
else:
    print("No places found.")

# Save the details to a JSON file
save_to_json(places)

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 25.5569837, 'lng': -80.3766892}, 'viewport': {'northeast': {'lat': 25.55824457989272, 'lng': -80.37533527010727}, 'southwest': {'lat': 25.55554492010728, 'lng': -80.37803492989272}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png', 'icon_background_color': '#7B9EB0', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet', 'name': 'Surgery Recovery House LLC after Lipo Bbl Tummy', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112690085055368390851">A Google User</a>'], 'photo_reference': 'AWYs27xB6bE6LFTkRE4tlHLVIKGc-_ksE8eu68uNBtYsZ3dqEaXVimSkzUdkH3eKQKbVUyEH1Bf3wPUJz_IAv1Qn-6KGUt0-3jqL7cy82UDRZoaSVqsDGGN9jO6vQVunbrucsUs2P8rlXU1tfBUpBNQAFMt0A-aTJuB2FarccnHqjiYrUXry', 'width': 3024}], 'place_id': 'ChIJDd23sFDD2YgR